In [4]:

from datetime import datetime
import functools
import os

from IPython.display import HTML, clear_output

import gymnasium as gym
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

try:
  import brax
except ImportError:
  !pip install git+https://github.com/google/brax.git@main
  clear_output()
  import brax

from brax import envs
from brax import jumpy as jp
from brax.io import html
from brax.io import model
from brax.training.agents.ppo import train as ppo
from brax.training.agents.sac import train as sac

if 'COLAB_TPU_ADDR' in os.environ:
  from jax.tools import colab_tpu
  colab_tpu.setup_tpu()

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### Enviornment

In [5]:
env = gym.make('FetchSlide-v2', max_episode_steps=10)

In [6]:
env

<TimeLimit<OrderEnforcing<PassiveEnvChecker<MujocoFetchSlideEnv<FetchSlide-v2>>>>>